In [ ]:
# Standard library
import json
import os
from collections import Counter, defaultdict
from collections.abc import Callable
from hashlib import sha256
from pathlib import Path
from typing import Any, Never
import collections.abc

import matplotlib.pyplot as plt

# Third-party libraries
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed
from matplotlib.ticker import PercentFormatter
from networkx import DiGraph
from rich.console import Console
from rich.progress import track

from ariel.body_phenotypes.robogen_lite.config import (
    NUM_OF_FACES,
    NUM_OF_ROTATIONS,
    NUM_OF_TYPES_OF_MODULES,
)
from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import (
    HighProbabilityDecoder,
)
from ariel.body_phenotypes.robogen_lite.decoders.visualize_tree import (
    visualize_tree_from_graph,
)

# Local libraries
from ariel.body_phenotypes.robogen_lite.modules.brick import BRICK_MASS
from ariel.body_phenotypes.robogen_lite.modules.core import CORE_MASS
from ariel.body_phenotypes.robogen_lite.modules.hinge import (
    ROTOR_MASS,
    STATOR_MASS,
)

console = Console()

In [ ]:
# Global constants
SCRIPT_NAME = "Initial"
CWD = Path.cwd()
DATA = Path(CWD / "__data__" / SCRIPT_NAME)
DATA.mkdir(exist_ok=True)
SEED = 40

# Global functions
console = Console()
RNG = np.random.default_rng(SEED)


In [ ]:
def generate_body(num_modules: int = 20) -> DiGraph:
    """Generate random robot with HighProbabilityDecoder."""

    # "Type" probability space
    type_probability_space = RNG.random(
        size=(num_modules, NUM_OF_TYPES_OF_MODULES),
        dtype=np.float32,
    )

    # "Connection" probability space
    conn_probability_space = RNG.random(
        size=(num_modules, num_modules, NUM_OF_FACES),
        dtype=np.float32,
    )

    # "Rotation" probability space
    rotation_probability_space = RNG.random(
        size=(num_modules, NUM_OF_ROTATIONS),
        dtype=np.float32,
    )

    # Decode the high-probability graph
    hpd = HighProbabilityDecoder(num_modules)
    hpd.probability_matrices_to_graph(
        type_probability_space,
        conn_probability_space,
        rotation_probability_space,
    )

    # Decode the high-probability graph
    hpd = HighProbabilityDecoder(num_modules)
    return hpd.probability_matrices_to_graph(
        type_probability_space,
        conn_probability_space,
        rotation_probability_space,
    )

In [ ]:
# def run(
#     robot: CoreModule,
#     *,
#     with_viewer: bool = False,
# ) -> None:
#     """Entry point."""
#     # MuJoCo configuration
#     viz_options = mujoco.MjvOption()  # visualization of various elements

#     # Visualization of the corresponding model or decoration element
#     viz_options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
#     viz_options.flags[mujoco.mjtVisFlag.mjVIS_ACTUATOR] = True
#     viz_options.flags[mujoco.mjtVisFlag.mjVIS_BODYBVH] = True

#     # MuJoCo basics
#     world = TiltedFlatWorld()

#     # Set random colors for geoms
#     for i in range(len(robot.spec.geoms)):
#         robot.spec.geoms[i].rgba[-1] = 0.5

#     # Spawn the robot at the world
#     world.spawn(robot.spec)

#     # Compile the model
#     model = world.spec.compile()
#     data = mujoco.MjData(model)

#     # Save the model to XML
#     xml = world.spec.to_xml()
#     with (DATA / f"{SCRIPT_NAME}.xml").open("w", encoding="utf-8") as f:
#         f.write(xml)

#     # Number of actuators and DoFs
#     console.log(f"DoF (model.nv): {model.nv}, Actuators (model.nu): {model.nu}")

#     # Reset state and time of simulation
#     mujoco.mj_resetData(model, data)

#     # Render
#     # single_frame_renderer(model, data, steps=10)

#     # View
#     if with_viewer:
#         viewer.launch(model=model, data=data)

In [ ]:
# delta = 10000  # just a random nr that bigger than 0.1
# diversity = 0
# nr_of_blocks = []
# old_avg_b = 0
# old_avg_h = 0
# nr_of_hinges = []
# learnability = 0  # maybe later
# graph_list = []
# while delta > 0.00001:
#     graph = generate_body()
#     graph_list.append(graph)
#     graph = dict(graph.nodes)
#     hinges = sum(graph[node]["type"] == "HINGE" for node in graph)
#     blocks = sum(graph[node]["type"] == "BRICK" for node in graph)
#     nr_of_blocks.append(blocks)
#     nr_of_hinges.append(hinges)

#     # skip the first 100
#     if len(nr_of_blocks) < 100:
#         continue

#     # go until we broke
#     new_avg_b = np.average(nr_of_blocks)
#     new_avg_h = np.average(nr_of_hinges)
#     delta = max(abs(old_avg_b - new_avg_b), abs(old_avg_h - new_avg_h))

#     if len(graph_list) >= 100_000:
#         break

#     old_avg_b = new_avg_b
#     old_avg_h = new_avg_h

# print(
#     f"Done!\nEnded with a delta of {max(abs(old_avg_b - new_avg_b), abs(old_avg_h - new_avg_h))}\nThe avg nr of blocks are: {new_avg_b}\nThe avg nr of hinges is: {new_avg_h}\nConcluded after {len(graph_list)} generated robots!"
# )


---

### Functions for analysis on individual

In [ ]:
def analyze_module_counts(individual: DiGraph) -> dict[str, int]:
    counts: dict[str, int] = {
        "core": sum(
            data["type"] == "CORE" for _, data in individual.nodes(data=True)
        ),
        "brick": sum(
            data["type"] == "BRICK" for _, data in individual.nodes(data=True)
        ),
        "hinge": sum(
            data["type"] == "HINGE" for _, data in individual.nodes(data=True)
        ),
        "none": sum(
            data["type"] == "NONE" for _, data in individual.nodes(data=True)
        ),
        "edges": len(individual.edges()),
    }
    counts["not-none"] = counts["core"] + counts["brick"] + counts["hinge"]
    assert counts["core"] == 1
    return counts

In [ ]:
def analyze_mass(individual: DiGraph) -> dict[str, float]:
    counts = analyze_module_counts(individual)
    core_mass = counts["core"] * CORE_MASS
    brick_tot_mass = counts["brick"] * BRICK_MASS
    hinge_tot_mass = counts["hinge"] * (ROTOR_MASS + STATOR_MASS)
    total_mass = core_mass + brick_tot_mass + hinge_tot_mass
    return {"mass": total_mass}

In [ ]:
def analyze_json_hash(individual: DiGraph) -> dict[str, str]:
    """Return a SHA-256 hash of a deterministic representation of the graph."""
    # Collect nodes and edges with attributes, sorted for determinism
    nodes = sorted([(n, dict(individual.nodes[n])) for n in individual.nodes()])
    edges = sorted([
        (u, v, dict(individual.edges[u, v])) for u, v in individual.edges()
    ])
    # Create a canonical dict
    canonical = {"nodes": nodes, "edges": edges}
    # Hash the canonical JSON string
    hash_string = sha256(
        json.dumps(canonical, sort_keys=True).encode("utf-8"),
    ).hexdigest()
    return {"hash": hash_string}

### Parallel running method for individual analysis data

In [ ]:
def analyze_individual(
    analyzers: Callable[[DiGraph], dict[str, Any]], num_modules=None,
):
    individual = generate_body(num_modules) if num_modules else generate_body()
    results = {"population": [individual]}
    for analyzer in analyzers:
        results.update(analyzer(individual))
    return results


def analyze_population(
    population_size: int,
    *,
    num_modules: int | None = None,
    analyzers: Callable[[DiGraph], dict[str, Any]] | None = None,
    n_jobs: int = -1,
) -> tuple[dict[str, Any], list[DiGraph]]:
    if analyzers is None:
        analyzers = [analyze_module_counts, analyze_mass, analyze_json_hash]

    if n_jobs != 1:
        raise NotImplementedError

    if n_jobs == -1:
        available_cpus = os.cpu_count()
        if available_cpus is not None and available_cpus > 2:
            n_jobs = available_cpus - 2
        elif available_cpus is not None and available_cpus > 1:
            n_jobs = available_cpus - 1
        else:
            n_jobs = 1

    console.log(f"Using {n_jobs} cpu(s)")
    console.log(f"Starting analysis of {population_size} individuals...")

    # Parallel execution with progress bar
    with Parallel(n_jobs=n_jobs) as parallel:
        results = parallel(
            delayed(analyze_individual)(analyzers)
            for _ in track(
                range(population_size),
                description="Analyzing population",
            )
        )

    properties_dict = defaultdict(list)
    # cast(results, dict)
    for result in results:
        for key, value in result.items():
            properties_dict[key].append(value)

    console.log("Analysis complete.")
    final_dict = dict(properties_dict)
    population = final_dict.pop("population")
    return final_dict, population

### Visualization methods for the analysis on a population

In [ ]:
def create_boxplot_from_dict(
    properties_dict: dict[str, Any],
    keys: list[str] | None = None,
) -> None:
    # If no keys provided, plot all except 'population'
    if keys is None:
        keys = list(properties_dict.keys())
    # Prepare data for seaborn
    data = []
    labels = []
    for key in keys:
        if key in properties_dict:
            data.append(properties_dict[key])
            labels.append(key)
    # Create boxplot
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=data)
    plt.xticks(ticks=range(len(labels)), labels=labels)
    plt.ylabel("Value")
    plt.title("Boxplot of Robot Properties")
    plt.show()

In [ ]:
def create_histogram_from_dict(
    properties_dict: dict[str, Any],
    keys: list[str] | None = None,
    bins: int = 30,
) -> None:
    """
    Draw a grouped histogram of the selected keys.
    For string-valued keys (e.g., hashes), plot the distribution of their frequencies (separately).
    For numeric keys, plot all together in one histogram.
    Title is the key names if there are fewer than 3, otherwise "Histogram of Robot Properties".
    """
    if keys is None:
        keys = list(properties_dict.keys())

    valid_keys = [k for k in keys if k in properties_dict]

    # Check if all selected keys are numeric
    if all(isinstance(properties_dict[k][0], (int, float)) for k in valid_keys):
        # Plot all numeric keys together
        data = [properties_dict[k] for k in valid_keys]
        plt.figure(figsize=(10, 6))
        n, bins_, _patches = plt.hist(
            data,
            bins=bins,
            label=valid_keys,
            edgecolor="black",
            linewidth=1.2,
            alpha=0.8,
            histtype="bar",
            rwidth=0.9,
        )
        total = n.sum()
        plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=total))
        plt.xlabel("Value")
        plt.ylabel("Frequency")
        plt.legend()
        plt.title(
            ", ".join(valid_keys)
            if len(valid_keys) <= 3
            else "Histogram of Robot Properties",
        )
        plt.show()
    else:
        # Plot string-valued keys separately
        for key in valid_keys:
            values = properties_dict[key]
            if not values:
                continue
            plt.figure(figsize=(10, 6))
            if isinstance(values[0], str):
                counts = Counter(values)
                freqs = list(counts.values())
                n, bins_, _patches = plt.hist(
                    freqs,
                    bins=bins,
                    edgecolor="black",
                    linewidth=1.2,
                    alpha=0.8,
                    rwidth=0.9,
                    label=[key],
                    histtype="bar",
                )
                total = n.sum()
                plt.gca().yaxis.set_major_formatter(
                    PercentFormatter(xmax=total),
                )
                plt.xlabel("Genotype frequency")
                plt.ylabel("Number of unique genotypes")
                plt.title(key)
                plt.legend()
                plt.show()

In [ ]:
def statistical_df_from_dict(
    properties_dict: dict[str, Any],
    keys: list[str] | None = None,
    save_file: Path | str | None = None,
) -> pd.DataFrame:
    """
    For each key in properties_dict:
      - If numeric: mean, std, median, count, Q1, Q3, outlier indexes, num_outliers, uniques.
      - If non-numeric: key, count, uniques.
    Returns a DataFrame with correct dtypes.
    """
    if keys is None:
        keys = list(properties_dict)

    rows = []
    for key in keys:
        values = properties_dict[key]
        if not values:
            continue
        uniques = len(set(values))
        if isinstance(values[0], (float, int)):
            arr = np.asarray(values, dtype=float)
            mean = arr.mean()
            std = arr.std(ddof=1)
            median = np.median(arr)
            q1 = np.percentile(arr, 25)
            q3 = np.percentile(arr, 75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            outlier_indexes = np.where((arr < lower) | (arr > upper))[
                0
            ].tolist()
            row = {
                "key": key,
                "count": len(arr),
                "uniques": int(uniques),
                "mean": float(mean),
                "std": float(std),
                "median": float(median),
                "Q1": float(q1),
                "Q3": float(q3),
                "num_outliers": len(outlier_indexes),
                "outlier_indexes": outlier_indexes,
            }
        else:
            row = {
                "key": key,
                "count": len(values),
                "uniques": int(uniques),
                "mean": None,
                "std": None,
                "median": None,
                "Q1": None,
                "Q3": None,
                "num_outliers": None,
                "outlier_indexes": None,
            }
        rows.append(row)

    df = pd.DataFrame.from_records(rows)
    df = df.set_index("key")

    # Set dtypes explicitly
    df["count"] = df["count"].astype("Int64")
    df["uniques"] = df["uniques"].astype("Int64")
    for col in ["mean", "std", "median", "Q1", "Q3"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["num_outliers"] = df["num_outliers"].astype("Int64")
    df["outlier_indexes"] = df["outlier_indexes"].astype(object)

    if save_file:
        df.to_csv(save_file)

    return df

In [ ]:
def count_df_from_list(value_list: list[str]) -> pd.DataFrame:
    """
    Returns a DataFrame with:
      - index: hash value
      - count: number of times the hash appears
      - indexes: list of indexes in the original list where the hash appears.
    """
    index_dict = defaultdict(list)
    for idx, h in enumerate(value_list):
        index_dict[h].append(idx)

    results = []
    for h, idxs in index_dict.items():
        results.append({
            "value": h,
            "count": len(idxs),
            "indexes": idxs,
        })

    df = pd.DataFrame(results)
    return df.set_index("value")

---

### Running the experiment

In [ ]:
# set n_jobs = 1 to not use parallelization. MUST be used rn
properties_dict, population = analyze_population(100_000, n_jobs=1)

In [ ]:
create_boxplot_from_dict(properties_dict)

create_histogram_from_dict(properties_dict, keys=["brick", "hinge", "none"])
create_histogram_from_dict(properties_dict, keys=["not-none"])
create_histogram_from_dict(properties_dict, keys=["mass"])
create_histogram_from_dict(properties_dict, keys=["edges"])
create_histogram_from_dict(properties_dict, keys=["hash"])

In [ ]:
stats_df = statistical_df_from_dict(properties_dict)
console.print(stats_df)

console.rule(style="rule.line dim")

# To get the unique counts per property
count_df = count_df_from_list(properties_dict["edges"])
console.print(count_df)

---

### Visualize Outliers Interactively

Visualize individuals from a key value dictionary where value contains indexes of the individuals from the population.

In [ ]:
# class IndividualVisualizer:
#     def __init__(
#         self,
#         index_dict: dict,
#         population: list,
#         console=None,
#     ) -> None:
#         """
#         index_dict: dict mapping keys to lists of individual indexes (may be empty).
#         population: list of individuals (e.g., DiGraph objects).
#         console: rich.console.Console instance for printing (optional).
#         """
#         # self._index_dict = {
#         #     k: v
#         #     for k, v in index_dict.items()
#         #     if isinstance(v, (list, tuple, set))
#         #     and v  # only non-empty lists/tuples/sets
#         # }
#         self._dict = self._filter_nested_index_dict(index_dict)
#         self._keys = self._create_key_list(self._dict)
#         print(self._dict)
#         print(self._keys)
        
#         self._current_nested_idx = 0
#         self._nested_idx_dict = 0
        
#         self._population = population
#         # self._key_index = 0
#         self._individual_index = 0
#         self._direction = 1
#         self._console = console or Console()

#     def forward(self):
#         # if (self._nested_idx_dict
        
#         # only go forward if possible
        
#         self._nested_idx_dict[self._current_nested_idx] += 1
#         self._current_nested_idx += 1
#         self._nested_idx_dict[self._current_nested_idx] = 0
        
#     def backward(self):
#         # only go backwards if possible
        
#         self._nested_idx_dict.pop(self._current_nested_idx)
#         self._current_nested_idx -= 1
#         self._nested_idx_dict[self._current_nested_idx] -= 1

#     def retrieve_key(self):
#         keys, values = 
        
#         for key, value in self._nested_idx_dict.items():
#             that_dict = that_dict[list[value -1]]

#     def _filter_nested_index_dict(self, index_dict):
#         """
#         Recursively filters a dictionary, keeping only keys whose values are:
#         1. A nested dictionary that, after recursive filtering, is not empty.
#         2. A non-empty list, tuple, or set containing only integers.
        
#         Args:
#             index_dict (dict): The dictionary (potentially nested) to filter.

#         Returns:
#             dict: The filtered dictionary, preserving structure.
#         """
#         if not isinstance(index_dict, collections.abc.Mapping):
#             raise TypeError

#         filtered_dict = {}
#         for k, v in index_dict.items():
#             if isinstance(v, collections.abc.Mapping):
#                 # Case 1: Nested dictionary.  and keep if the result is non-empty.
#                 filtered_v = self._filter_nested_index_dict(v)
#                 if filtered_v:
#                     filtered_dict[k] = filtered_v
#             else:
#                 # Case 2: Leaf node. Check if it's a non-empty list/tuple/set of ints.
#                 is_valid_list = (
#                     isinstance(v, (list, tuple, set)) and 
#                     v and 
#                     all(isinstance(item, int) for item in v)
#                 )
                
#                 if is_valid_list:
#                     filtered_dict[k] = v
                
#         return filtered_dict

#     def _create_key_list(self, dict_of_dicts: dict) -> list:
#         # if not isinstance(data, collections.abc.Mapping):
#         #     return []
#         result = []
#         for key, value in dict_of_dicts.items():
#             result.append(key)
#             if isinstance(value, collections.abc.Mapping):
#                 nested_keys_list = self._create_key_list(value)
#                 result.append(nested_keys_list)
#         return result 

#     def get_count_from_index(self) -> Never:
#         """
#         Would be cool to select the counts? so switch from main_key to count_key?
#         maybe the index_dict is just smart and if it contains a dict, it will createa a sub key selection method??
#         """
#         raise NotImplementedError

#     def select_key(self, index=None) -> None:
#         if not self._keys:
#             self._console.print("[red]No keys with individuals found.[/red]")
#             return

#         if index is not None:
#             if 0 <= index < len(self._keys):
#                 self._key_index = index
#                 self._individual_index = 0

#             self._console.print("[red]Index for key out of bounds[/red]")
#             return
#         self._key_index = (self._key_index + self._direction) % len(
#             self._keys,
#         )
#         self._individual_index = 0

#         index_count = [len(self._dict[key]) for key in self._keys]
#         selected_key = self._keys[self._key_index]
#         selected_count = index_count[self._key_index]
#         key_lines = []
#         display_key = selected_key
#         if isinstance(selected_key, str):
#             max_len = 20
#             display_key = (
#                 (selected_key[:max_len] + "…")
#                 if len(selected_key) > max_len
#                 else selected_key
#             )
#         key_lines.append(
#             f"[bold underline]Selected: {self._key_index}/{len(self._keys)} | {display_key} ({selected_count} indexes)[/bold underline]\n",
#         )
#         for i, (key, total_indexes) in enumerate(
#             zip(self._keys, index_count, strict=False),
#         ):
#             if i == self._key_index:
#                 key_lines.append(
#                     f"[bold green]{i:>3} > {key} <[/bold green] [yellow]({total_indexes} indexes)[/yellow]",
#                 )
#             else:
#                 key_lines.append(
#                     f"{i:>3} {key} [dim]({total_indexes} individuals)[/dim]",
#                 )
#         self._console.print("\n".join(key_lines))

#     def _print_direction(self) -> None:
#         arrow = "→" if self._direction == 1 else "←"
#         self._console.print(
#             f"[cyan]Direction: {'forward' if self._direction == 1 else 'backward'} {arrow}[/cyan]",
#         )

#     def toggle_direction(self) -> None:
#         self._direction = -self._direction
#         arrow = "→" if self._direction == 1 else "←"
#         self._console.print(
#             f"[cyan]Direction toggled. Now: {'forward' if self._direction == 1 else 'backward'} {arrow}[/cyan]",
#         )

#     def cycle_through_individual_index(self):
#         indexes = self._get_index_list()
#         if indexes:
#             self._individual_index = (
#                 self._individual_index + self._direction
#             ) % len(indexes)
#         self._show_current_selection()
#         return self._get_current_individual_value()

#     def visualize_individual(
#         self,
#         cycle=True,
#         visualize_fn=visualize_tree_from_graph,
#     ) -> None:
#         """visualize_fn: function to visualize an individual, e.g. visualize_tree_from_graph."""
#         if cycle:
#             idx = self.cycle_through_individual_index()
#         else:
#             idx = self._get_current_individual_value()
#         if idx is not None and self._population:
#             visualize_fn(self._population[idx][0])
#         else:
#             self._console.print("[red]No individual to visualize.[/red]")

#     def _get_index_list(self):
#         if not self._keys:
#             return []
#         key = self._keys[self._key_index]
#         return self._dict.get(key, [])

#     def _show_current_selection(self) -> None:
#         if not self._keys:
#             self._console.print("[red]No keys with individuals found.[/red]")
#             return
#         key = self._keys[self._key_index]
#         indexes = self._get_index_list()
#         total = len(indexes)
#         if indexes:
#             idx = self._individual_index % total
#             progress_str = f"[{idx + 1}/{total}]"
#             self._console.print(
#                 f"[bold green]Key:[/bold green] {key} | "
#                 f"[blue]Individual index:[/blue] {idx} {progress_str} | "
#                 f"[magenta]Value:[/magenta] {indexes[idx]} | "
#                 f"[yellow]Total individuals:[/yellow] {total}",
#             )
#         else:
#             self._console.print(
#                 f"[bold green]Key:[/bold green] {key} | [red]No individuals[/red]",
#             )

#     def _get_current_individual_value(self):
#         indexes = self._get_index_list()
#         if indexes:
#             return indexes[self._individual_index % len(indexes)]
#         return None

In [ ]:
# Parsing the outliers in the visualizer
visualizer = IndividualVisualizer(
    stats_df["outlier_indexes"].to_dict(),
    population,
)

In [ ]:
# Parsing the outliers in the visualizer
visualizer = IndividualVisualizer(
    count_df["indexes"].to_dict(),
    population,
)

<!-- The exeperiment shows that the outliers seem to have the same genotype. This leads to the question for how many unique genotypes are actually being created. To run this experiment, a new function should be created that easily parses the given phenotype into a deterministic string. this can than be used in a histogram to count the unique types? -->


The visualizer shows that while the phenotype may be roughly the same, the genotypes are all unique. A new determinstic method needs to be invented to create a 1:1 genotype <-> phenotype canonical mapping.

so going from (many) genotype -> 1 canonical genotype -> 1 phenotype

with this new metric the follownig can be implemented

step 1: normalize the ID's. and check again

---

### To be implemented